# Generate Spread using dedicated service - C#

### Overview
Spread indicator is commonly used to provide midpoint value from two given inputs. 

Spread is generally impacted by two major factors: 
* *Volatility*: the more volatile an asset, the higher the spread
* *Liquidity*: the more liquid an asset, the lower the spread

When correlated with a range of indicators, it gives an idea about the market quality.

This sample demonstrates how to request and plot the spread from a dedicated data service.

### Inputs/outputs
Spread indicator requires instrument's identifier and date time intervals as per inputs and returns the spread data points.

For each timestamp, a spread data point is the difference between the bid and the ask prices. 

This sample shows how to plot a simple spread graph for basis technical analysis using an open source library. 

### Services used
This sample uses *gRPC requests* in order to retrieve top of the book tick data from the hosted service. The queried endpoint in this script are:
* *TickBookService*: to directly retrieve best Bid/Ask tick data from the server.

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Tick.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run Spread sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis"
#r "nuget: XPlot.Plotly.Interactive"

In [ ]:
using Systemathics.Apis.Helpers;
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.Tick.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Core;
using Grpc.Net.Client;
using XPlot.Plotly;
using XPlot.Plotly.Interactive;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
var headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
var channel = ChannelHelpers.GetChannel();

### Step 3: Retrieve data
To request *tick data* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Tick data request parameters

#### 3.1 Instrument selection

In [ ]:
var ticker = "AAPL";
var exchange = "BATS";

#### 3.2 Time period delimitation

In [ ]:
// Create time intervals
var start = new DateTime(2021, 03, 05);
var end = new DateTime(2021, 03, 05);
var dateIntervals = new DateInterval()
{
  StartDate = new Date { Year = start.Year, Month = start.Month, Day = start.Day },
  EndDate = new Date { Year = end.Year, Month = end.Month, Day = end.Day }
};

// Build the market data request time interval (we are using Google date time format : we have to cast the dates)
// UTC time zone
var timeInterval = new TimeInterval()
{
 StartTime = new TimeOfDay { Hours = 14, Minutes = 00, Seconds = 00 },
 EndTime = new TimeOfDay { Hours = 14, Minutes = 30, Seconds = 00 } 
};

#### 3.3 Request creation
The following code snippets creates *gRPC client*, process request and returns the request reply:

In [ ]:
// Generate constraints based on the previous time selection:
var constraints = new Constraints(); 
constraints.DateIntervals.Add(dateIntervals);
constraints.TimeIntervals.Add(timeInterval);

In [ ]:
// Generate the tick book request
var request = new TickBookRequest { Constraints = constraints };
var identifier = new Identifier { Exchange = exchange, Ticker = ticker};
request.Identifiers.Add(new [] {identifier}); 

In [ ]:
// Instantiate the tick book service
var service = new TickBookService.TickBookServiceClient(channel);

### Step 4: Visualize data

#### 4.1 Retrieve Spread data

In [ ]:
// Get the limits
var limits = new List<BookLimit>();
var call = service.TickBook(request, headers);
await foreach (var book in call.ResponseStream.ReadAllAsync())
{
    if (book.Limit != null) 
    {
        limits.Add(book.Limit);
    } 
}

// Display top of the book
//display(limits);

#### 4.2 Plot Spread

Display *spread* over the look back period with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package.
Plot a simple line graphs for *spread* as follows:

In [ ]:
var spread = new Scattergl
{
    name = "Spread",
    mode = "lines",
    x = limits.Select(elt => elt.TimeStamp.ToDateTime()),
    y = limits.Select(elt => elt.Ask.Price - elt.Bid.Price)
};
var chart = Chart.Plot(spread);
chart.WithTitle($"Spread for {ticker}");
chart.WithXTitle($"Time");
chart.WithYTitle($"Price");
chart.Width = 1500;
chart.Height = 800;
display(chart);